In [5]:
import pandas as pd

FILE = "/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv"
SIZE = 20000
VIRAL_THRESHOLD = 25

data = pd.read_csv(FILE)
data = data.sample(SIZE)

In [6]:
data = data[['title', 'num_points']]

In [7]:
viral = data["num_points"].apply(lambda x: pd.Series(1 if x >= VIRAL_THRESHOLD else 0))
viral.columns=["viral"]

In [8]:
#skew checking
viral.describe()

,viral
count,20000.000000
mean,0.099900
std,0.299874
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
from nltk.tokenize import StanfordTokenizer
from nltk.tag import StanfordPOSTagger

st = StanfordPOSTagger('english-bidirectional-distsim.tagger')

import numpy as np
title = data["title"].apply(lambda x: pd.Series([y[1] for y in st.tag(StanfordTokenizer().tokenize(x))]))
title

In [ ]:
dic = {};

def get_mapping(token, dic):
    if not dic.get(token):
        dic[token] = len(dic);
    return dic[token]

title_emb = title.applymap(lambda x: get_mapping(x, dic) )
print(dic)
title_emb

In [ ]:
hn_data = viral
for i in range(len(title_emb)):
    hn_data = hn_data.join(title_emb[i])

In [ ]:
testing_data = hn_data.sample(frac=0.2)
training_data = hn_data.drop(testing_data.index)

testing_data.to_csv('./hn_testing_pos_%d_%d.csv' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)
training_data.to_csv('./hn_training_pos_%d_%d.csv' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)